In [1]:
# Swinburne uni Machine learning Course final project
# creator: Qiyuan zhu
# Initalized: 28-03-2021
# Project: Actiong recognition in knowledge distilling

import os
import sys
import numpy as np
import pandas as pd
import Data_processing as Dp
import GlobalSelector as Glo
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

strategy = tf.distribute.MirroredStrategy()

print ("#" * 100)
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))
print ("#" * 100)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
####################################################################################################
Number of devices: 1
####################################################################################################


### 1.0 Read data

In [2]:
# Video options
_INPUT_SIZE = 224
_NUM_FRAMES = 64


In [3]:
record_base_path = "C:\\Users\\laiho\\Desktop\\machine_learning_project-cos80028\\UCF\\UCF_final_testing"

In [4]:
video_list, label = Dp.load_dataset(record_base_path)

### 2.0 generate feature 

In [5]:
raw_input, raw_label = Dp.data_load(video_list, label)

(1280, 1, 224, 224, 3)


In [6]:
single_selector_input = Dp.feature_extraction(raw_input)
single_selector_input = tf.convert_to_tensor(single_selector_input)


In [7]:
global_selector_pair_input = Dp.random_pair_generation(single_selector_input)  # train X

(1280, 1, 6000) prepared_pair


In [8]:
print(global_selector_pair_input)

tf.Tensor(
[[[ 3.9047253e-04  7.9621671e-04  2.2631395e-03 ... -5.7218000e-03
    1.6466001e-01 -3.9074001e-01]]

 [[ 4.3815366e-04  1.7802168e-04  6.2966318e-04 ... -5.7218000e-03
    1.6466001e-01 -3.9074001e-01]]

 [[ 3.6820435e-04  6.6704740e-04  1.1037615e-03 ... -5.7218000e-03
    1.6466001e-01 -3.9074001e-01]]

 ...

 [[ 1.7245753e-03  5.4329156e-04  1.3617958e-03 ... -5.7218000e-03
    1.6466001e-01 -3.9074001e-01]]

 [[ 2.2686922e-03  1.0179336e-03  3.8820758e-04 ... -5.7218000e-03
    1.6466001e-01 -3.9074001e-01]]

 [[ 1.2774186e-03  1.5165649e-03  9.3110104e-04 ... -5.7218000e-03
    1.6466001e-01 -3.9074001e-01]]], shape=(1280, 1, 6000), dtype=float32)


In [9]:
final_label = Dp.get_list_label(raw_label)
print(final_label.shape)   # train Y

(1280,)


In [10]:
final_label2= pd.get_dummies(final_label)

### 4.0 model

In [12]:
model = Glo.create_global_frame_selector(num_class = 10)

print(model.summary())

omega (None, 1024)
hi (None, 1024)
lambda (None, 1024)
gama (None, 1024)
ct (None, 1024)
representation (None, 1024)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1, 6000)]    0                                            
__________________________________________________________________________________________________
flatten_2561 (Flatten)          (None, 6000)         0           input_3[0][0]                    
__________________________________________________________________________________________________
dense_1280 (Dense)              (None, 1024)         6145024     flatten_2561[0][0]               
__________________________________________________________________________________________________
alpha_i (Dense)                 (None, 1024)         1049600     dense_1280[

In [13]:

    #model = Glo.create_global_frame_selector(num_class = len(set(label)))
    #print(model.summary())

    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        "global_selector.h5",
        monitor="accuracy",
        save_best_only=True,
        #save_weights_only=True,
    )
    #es_callback = EarlyStopping(monitor='val_loss', patience=5)
    
    history = model.fit(
        x=tf.convert_to_tensor(global_selector_pair_input),
        y=tf.convert_to_tensor(final_label2),
        batch_size=32,
        epochs=100,
        #validation_split=0.3,
        verbose = 2,
        callbacks=[checkpoint_callback]
    )
    hist_df = pd.DataFrame(history.history) 
    #result = ["Evaluation result: ","loss: "+ str(loss), "accuracy: "+ str(accuracy), "elapsed_time: "+ str(elapsed_time)]
    #result = pd.DataFrame(result)
    with open('global_selector_history.csv', mode='w') as f:
        #result.to_csv(f)
        hist_df.to_csv(f)

Epoch 1/100
40/40 - 6s - loss: 1.8644 - accuracy: 0.3391
Epoch 2/100
40/40 - 6s - loss: 1.3475 - accuracy: 0.4203
Epoch 3/100
40/40 - 5s - loss: 1.3055 - accuracy: 0.4328
Epoch 4/100
40/40 - 5s - loss: 1.2814 - accuracy: 0.4469
Epoch 5/100
40/40 - 5s - loss: 1.2591 - accuracy: 0.4359
Epoch 6/100
40/40 - 5s - loss: 1.2452 - accuracy: 0.4672
Epoch 7/100
40/40 - 5s - loss: 1.2540 - accuracy: 0.4688
Epoch 8/100
40/40 - 5s - loss: 1.2391 - accuracy: 0.4586
Epoch 9/100
40/40 - 5s - loss: 1.2377 - accuracy: 0.4664
Epoch 10/100
40/40 - 6s - loss: 1.2438 - accuracy: 0.4391
Epoch 11/100
40/40 - 5s - loss: 1.2364 - accuracy: 0.4594
Epoch 12/100
40/40 - 5s - loss: 1.2352 - accuracy: 0.4586
Epoch 13/100
40/40 - 6s - loss: 1.2313 - accuracy: 0.4625
Epoch 14/100
40/40 - 6s - loss: 1.2186 - accuracy: 0.4859
Epoch 15/100
40/40 - 5s - loss: 1.2303 - accuracy: 0.4508
Epoch 16/100
40/40 - 5s - loss: 1.2260 - accuracy: 0.4664
Epoch 17/100
40/40 - 6s - loss: 1.2196 - accuracy: 0.4625
Epoch 18/100
40/40 - 6s

In [14]:
    model.predict(global_selector_pair_input)

array([[ 3.9856553 ,  4.071913  , -2.7884288 , ..., -2.9726572 ,
        -0.45769164, -2.0645354 ],
       [ 3.9855614 ,  4.0718265 , -2.7884398 , ..., -2.9726481 ,
        -0.4576096 , -2.0642607 ],
       [ 3.985838  ,  4.072054  , -2.7884817 , ..., -2.9727323 ,
        -0.4578036 , -2.0652378 ],
       ...,
       [ 2.7322524 ,  2.180417  , -5.463234  , ..., -2.378465  ,
        -1.9420077 , 37.14533   ],
       [ 2.7315967 ,  2.1798713 , -5.4641457 , ..., -2.3791642 ,
        -1.9432226 , 37.142628  ],
       [ 4.4068294 ,  3.801651  , -6.351592  , ..., -3.72792   ,
        -1.5717124 , 35.152416  ]], dtype=float32)